In [106]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;

In [107]:
import os
os.chdir('C:/Users/DHRUVIL/OneDrive/Stratsntools/Python Directory')
df=pd.read_csv('abcnews-date-text.csv',error_bad_lines=False)

In [108]:
df.head()
#We have two columns: Publishing date and the headline text

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [109]:
#For our purposes, we'll focus on the headline text
data_text = df[['headline_text']].astype('str')

In [110]:
data_text.shape
#It's huge. We will not take all the headlines

(1103665, 1)

In [111]:
data_text=data_text.loc[1:100000,:]

In [112]:
data_text.shape

(100000, 1)

In [117]:
stopw=stopwords.words('english')
stopw
def stopwords_remove(x):
    terms=x.split()
    terms=[w for w in terms if w not in stopw]
    sentence=' '.join(terms)
    return sentence
data_text['Refined_headlines']=data_text['headline_text'].apply(lambda x: stopwords_remove(x))

In [118]:
data_text.head()

,headline_text,Refined_headlines
1,act fire witnesses must be aware of defamation,act fire witnesses must aware defamation
2,a g calls for infrastructure protection summit,g calls infrastructure protection summit
3,air nz staff in aust strike for pay rise,air nz staff aust strike pay rise
4,air nz strike to affect australian travellers,air nz strike affect australian travellers
5,ambitious olsson wins triple jump,ambitious olsson wins triple jump


In [121]:
headline_sentences=[''.join(text) for text in data_text['Refined_headlines']]

In [122]:
headline_sentences

['act fire witnesses must aware defamation',
 'g calls infrastructure protection summit',
 'air nz staff aust strike pay rise',
 'air nz strike affect australian travellers',
 'ambitious olsson wins triple jump',
 'antic delighted record breaking barca',
 'aussie qualifier stosur wastes four memphis match',
 'aust addresses un security council iraq',
 'australia locked war timetable opp',
 'australia contribute 10 million aid iraq',
 'barca take record robson celebrates birthday',
 'bathhouse plans move ahead',
 'big hopes launceston cycling championship',
 'big plan boost paroo water supplies',
 'blizzard buries united states bills',
 'brigadier dismisses reports troops harassed',
 'british combat troops arriving daily kuwait',
 'bryant leads lakers double overtime win',
 'bushfire victims urged see centrelink',
 'businesses prepare terrorist attacks',
 'calleri avenges final defeat eliminate massu',
 'call ethanol blend fuel go ahead',
 'carews freak goal leaves roma ruins',
 'cemete

In [124]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000);
x_counts = vectorizer.fit_transform(headline_sentences)

In [126]:
x_counts

<100000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 449491 stored elements in Compressed Sparse Row format>

In [127]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [128]:
x_tfidf

<100000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 449491 stored elements in Compressed Sparse Row format>

In [129]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [131]:
#number of topics
num_topics=10
#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd');
#fit the model
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=10, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [187]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict)

In [190]:
get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,police,us,man,govt,new,council,dies,plan,iraq,court
1,probe,iraqi,charged,urged,zealand,rejects,crash,water,says,face
2,investigate,baghdad,murder,nsw,laws,considers,car,back,war,charges
3,missing,killed,jailed,vic,hospital,land,woman,call,killed,faces
4,search,troops,missing,wa,president,backs,killed,restrictions,un,appeal
5,death,soldier,stabbing,qld,chief,seeks,hospital,group,troops,murder
6,shooting,open,hospital,sa,set,claims,accident,boost,bush,charge
7,seek,forces,sydney,funding,deal,merger,road,support,report,case
8,hunt,soldiers,killed,fire,york,approves,two,residents,howard,hears
9,fatal,attack,attack,boost,centre,consider,aged,concerns,pm,high
